In [2]:
import ee
ee.Initialize()

In [8]:
import pandas as pd
import numpy as np
import matplotlib
import scipy
import folium
import shapely
from IPython.display import Image
import geopandas as gpd
import rasterio
import pygeohydro
import datetime
from geemap.plot import geopandas_to_ee
import geemap.foliumap as geemap

In [4]:
meadows_df = gpd.read_parquet("./raw_data/ucdmeadows.parquet")
meadows_df.head()

,geometry,AREA_ACRE,STATE,ID,HUC12,OWNERSHIP,EDGE_COMPLEXITY,DOM_ROCKTYPE,VEG_MAJORITY,COKEY,...,uniqueID,MDW_DEM_SLOPE,ED_MIN_FStopo_Trails,ED_MIN_FStopo_Roads,POUR_POINT_LAT,POUR_POINT_LON,StreamSlopeGrade,MDW_CATCHMENT_AREA,SHAPE_Length,SHAPE_Area
0,"MULTIPOLYGON (((151112.968 -286518.399, 151121...",2.283849,CA,UCDSNM000005,180300020601,Private,1.368988,granodiorite,Conifer,13197316,...,1,12.157462,336.005951,593.969727,35.426085,-118.334596,0.156945,6.283005e+05,466.548680,9242.407193
1,"MULTIPOLYGON (((151691.550 -284984.748, 151675...",17.017459,CA,UCDSNM000007,180300020601,Private,1.303125,granodiorite,Conifer,13197209,...,2,3.148981,488.773987,10.000000,35.443429,-118.326901,0.042227,4.897827e+06,1212.264166,68867.214181
2,"MULTIPOLYGON (((154869.884 -284068.757, 154873...",153.904101,CA,UCDSNM000008,180300020601,Private,2.944442,granodiorite,Conifer,13197209,...,3,2.654701,0.000000,0.000000,35.444935,-118.316133,0.020900,1.284576e+07,8237.434596,622827.799550
3,"MULTIPOLYGON (((152628.282 -283415.658, 152638...",63.880979,CA,UCDSNM000010,180300020601,Sequoia National Forest,2.027670,granodiorite,Conifer,13197209,...,4,4.535124,0.000000,0.000000,35.454702,-118.321363,0.012879,3.845936e+07,3654.659953,258517.150924
4,"MULTIPOLYGON (((125728.516 -282621.361, 125735...",14.829520,CA,UCDSNM000012,180300030107,Sequoia National Forest,2.178631,granodiorite,Conifer,13197209,...,5,8.577726,582.408813,0.000000,35.464744,-118.612956,0.105395,1.322439e+06,1891.956159,60012.937537


In [175]:
study_area_huc12 = '180400090505'
#study_area_huc12 = '180201210605'
wbd = pygeohydro.watershed.WBD('huc12')
data = wbd.byids(field='huc12', fids=[study_area_huc12])
meadows = meadows_df.loc[meadows_df['HUC12'].isin([study_area_huc12])]
study_area_ee = geopandas_to_ee(data)
meadows_ee = geopandas_to_ee(meadows)

In [176]:
naip_images = ee.ImageCollection('USDA/NAIP/DOQQ')
naip_images = naip_images.filterBounds(study_area_ee)
naip_images = naip_images.filterDate('2000-01-01', '2022-12-31')
naip_images = naip_images.sort('system:time_start', False)
naip_images = naip_images.map(lambda image: image.clip(study_area_ee))
#naip_images = naip_images.map(lambda image: image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI')))
#naip_images = naip_images.map(lambda image: image.addBands(image.normalizedDifference(['G', 'N']).rename('NDWI')))
#naip_images = naip_images.map(lambda image: image.addBands(image.normalizedDifference(['R', 'G']).rename('NDRE')))
#naip_images = naip_images.map(lambda image: image.addBands(image.normalizedDifference(['N', 'G']).rename('NDGI')))
naip_images

Name,Description
R,Red
G,Green
B,Blue
N,Near infrared


In [177]:
# print all the dates and number of images for the images in the collection
def get_dates(collection):
    dates = collection.aggregate_array('system:time_start').getInfo()
    dates = [datetime.datetime.fromtimestamp(x/1000) for x in dates]
    year_month = [x.strftime('%Y-%m') for x in dates]
    return year_month

dates = get_dates(naip_images)
dates = pd.Series(dates)
dates.value_counts()

2022-08    9
2020-06    9
2018-09    9
2016-06    9
2014-06    9
2012-06    9
2009-06    9
2005-08    9
2010-07    8
2010-08    1
Name: count, dtype: int64

In [178]:
# given a year, mosaic all the images from that year
def mosaic_year(year):
    year_images = naip_images.filterDate(year + '-01-01', year + '-12-31')
    year_images = year_images.mosaic()
    return year_images

years = dates.str.slice(0, 4).unique()
years = [str(x) for x in years]
images = [mosaic_year(year) for year in years]
images = list(zip(years, images))
images

[('2022', <ee.image.Image at 0x7fc88f1bfc40>),
 ('2020', <ee.image.Image at 0x7fc88f1bf7c0>),
 ('2018', <ee.image.Image at 0x7fc88f1bfa60>),
 ('2016', <ee.image.Image at 0x7fc88f1bfca0>),
 ('2014', <ee.image.Image at 0x7fc88f1bf880>),
 ('2012', <ee.image.Image at 0x7fc88f1bf970>),
 ('2010', <ee.image.Image at 0x7fc88f1bf370>),
 ('2009', <ee.image.Image at 0x7fc8b138e1f0>),
 ('2005', <ee.image.Image at 0x7fc8b138e430>)]

In [166]:
Map = geemap.Map()
Map.centerObject(study_area_ee, 12)

naip_params = {'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255}
for year, image in images:
    Map.addLayer(image, naip_params, year)
Map.addLayer(meadows_ee, {'color': 'green', 'opacity': 0.5}, 'meadows')
Map.addLayerControl()
Map

KeyboardInterrupt: 

In [ ]:
#Map.save('./outputs/naip.html')

In [213]:
# prep input image
image = images[1][1]
image = image.addBands(image.normalizedDifference(['N', 'R']).rename('NDVI'))
image = image.addBands(image.normalizedDifference(['G', 'N']).rename('NDWI'))
image = image.addBands(image.normalizedDifference(['R', 'G']).rename('NDRE'))
image = image.addBands(image.normalizedDifference(['N', 'G']).rename('NDGI'))

# add slope
slope = ee.Terrain.slope(ee.Image('USGS/SRTMGL1_003'))
image = image.addBands(slope.rename('slope'))
image

In [214]:
sample_size = 8000
meadow_points = image.sample(
    region=meadows_ee,
    scale=1,
    numPixels=sample_size
)
non_meadow_points = image.sample(
    region=study_area_ee.geometry().difference(meadows_ee),
    scale=1,
    numPixels=sample_size*2
)

In [215]:
meadow_points = meadow_points.map(lambda feature: feature.set('class', 1))
non_meadow_points = non_meadow_points.map(lambda feature: feature.set('class', 0))
training_data = meadow_points.merge(non_meadow_points)
cart = ee.Classifier.smileCart().train(training_data, 'class', image.bandNames())
classified = image.classify(cart)
classified_reduced = classified.reduceNeighborhood(
    reducer=ee.Reducer.median(),
    kernel=ee.Kernel.square(5),
)

In [218]:
Map = geemap.Map()
Map.centerObject(study_area_ee, 12)
naip_params = {'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255}
Map.addLayer(image, naip_params, 'image')
Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['red', 'green']}, 'classified')
Map.addLayer(classified_reduced, {'min': 0, 'max': 1, 'palette': ['red', 'green'], 'opacity': 0.5}, 'classified_reduced')
Map.addLayer(meadows_ee, {'color': 'black', 'opacity': 1.0, 'fillOpacity': 0}, 'meadows')
Map